In [1]:
import sys
import enum 
from tkinter import CENTER
import random
import pygame 
from pygame.locals import *

pygame 2.1.2 (SDL 2.0.18, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
import numpy as np
from PIL import ImageGrab
from PIL import Image
import cv2 
import io
import os 
import time
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = (30, 30)
import seaborn as sns
import pandas as pd
import numpy as np
from random import randint
import os
%matplotlib inline 
from keras.models import model_from_json
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from keras.callbacks import TensorBoard
from collections import deque
import random
import pickle
import json
import keyboard
# from game import gamstate

In [3]:
vec = pygame.math.Vector2
ACC = 0.2
FRIC = -0.5
curator_red = pygame.image.load("resources/curater/redlight.png")
curator_green = pygame.image.load("resources/curater/greenlight.jpg")
curator_turning = pygame.image.load("resources/curater/turning.jpg")
RED_STATE_TIME_RANGE = (1000,6000)
GREEN_STATE_TIME_RANGE = (400,3000)
TURNING_STATE_TIME_RANGE = (150,400)

In [4]:
class Player(pygame.sprite.Sprite):
    def __init__(self,x,y,width,height):
        super().__init__()
        self.x = x
        self.y = y
        self.width = width 
        self.height = height 
        self.surf = pygame.Surface((width,height))
        self.surf.fill((128,255,40))
        self.rect = self.surf.get_rect(center = (x,y))
        self.pos = vec((x,y))
        self.rect.midbottom = self.pos 
        self.vel = vec(0,0)
        self.acc = vec(0,0)
    def move(self):
        self.acc = vec(0,0)
        pressed_keys = pygame.key.get_pressed()
        if pressed_keys[K_UP]:
            self.acc.y = ACC
        self.acc.y += self.vel.y * FRIC
        self.vel += self.acc
        self.pos -= self.vel + 0.5 * self.acc
        self.rect.midbottom = self.pos

    def display(self,displaye_surface):
        displaye_surface.blit(self.surf , self.rect)

    def agent_move(self):
        self.acc = vec(0,0)
        self.acc.y = ACC
        self.acc.y += self.vel.y * FRIC
        self.vel += self.acc
        self.pos -= self.vel + 0.5 * self.acc
        self.rect.midbottom = self.pos
    
    def agent_stop(self):
        self.rect.midbottom = self.pos

In [5]:
class FinishLine(pygame.sprite.Sprite):
    def __init__(self,x,y,width,height):
        super().__init__()
        self.surf = pygame.Surface((width,height))
        self.surf.fill((255,0,0))
        self.rect = self.surf.get_rect(center = (x,y))
    def display(self,display):
        display.blit(self.surf, self.rect)

    def check_game_won(self,player_pos):
        if player_pos < self.rect.top:
            return True

In [6]:
class GameState(enum.Enum):
    ACTIVE = 1
    WON = 2
    LOST = 3

In [7]:
class CuratorState(enum.Enum):
    RED = 1 
    GREEN = 2
    TURNING = 3

class Curator(pygame.sprite.Sprite):
    def __init__(self,x,y,width,height,player:Player):
        super().__init__()
        self.x = x
        self.y = y
        self.width = width 
        self.height = height
        self.surf = pygame.Surface((width,height))
        self.rect = self.surf.get_rect(center = (x,y))
        self.state = CuratorState.GREEN
        self.last_state = CuratorState.TURNING 
        self.state_time_start = pygame.time.get_ticks()
        self.state_wait_time = random.uniform(GREEN_STATE_TIME_RANGE[0],GREEN_STATE_TIME_RANGE[1])
        self.player = player
        self.last_player_pos_y = player.rect.bottom
    def update(self):
        time_elapsed = (pygame.time.get_ticks() - self.state_time_start)
        if time_elapsed > self.state_wait_time:
            if self.state == CuratorState.GREEN:
                self.last_state = CuratorState.GREEN
                self.state = CuratorState.TURNING
                self.state_wait_time = random.uniform(TURNING_STATE_TIME_RANGE[0], TURNING_STATE_TIME_RANGE[1])
            elif self.state == CuratorState.TURNING and self.last_state == CuratorState.GREEN:
                self.last_state = CuratorState.TURNING
                self.state = CuratorState.RED
                self.state_wait_time = random.uniform(RED_STATE_TIME_RANGE[0],RED_STATE_TIME_RANGE[1])
                self.last_player_pos_y = self.player.rect.bottom
            elif self.state == CuratorState.TURNING and self.last_state == CuratorState.RED:
                self.last_state = CuratorState.TURNING
                self.state = CuratorState.GREEN 
                self.state_wait_time = random.uniform(GREEN_STATE_TIME_RANGE[0],GREEN_STATE_TIME_RANGE[1])
            elif self.state == CuratorState.RED:
                self.last_state = CuratorState.RED
                self.state = CuratorState.RED 
                self.state = CuratorState.TURNING
                self.state_wait_time = random.uniform(TURNING_STATE_TIME_RANGE[0],TURNING_STATE_TIME_RANGE[1])
            self.state_time_start = pygame.time.get_ticks()

    def display(self,display):
        if self.state == CuratorState.GREEN:
            display.blit(curator_green,self.rect)
        elif self.state == CuratorState.TURNING:
            display.blit(curator_turning,self.rect)
        elif self.state == CuratorState.RED:
            display.blit(curator_red,self.rect)

    def detect_movement(self):
        if self.state == CuratorState.RED:
            if self.player.rect.bottom < self.last_player_pos_y:
                return True
        False

In [8]:
#game parameters
ACTIONS = 2 # possible actions: Move Forward , do nothing
GAMMA = 0.99 # decay rate of past observations original 0.99
OBSERVATION = 50000. # timesteps to observe before training
EXPLORE = 100000  # frames over which to anneal epsilon
FINAL_EPSILON = 0.0001 # final value of epsilon
INITIAL_EPSILON = 0.1 # starting value of epsilon
REPLAY_MEMORY = 50000 # number of previous transitions to remember
BATCH = 32 # size of minibatch
FRAME_PER_ACTION = 1
LEARNING_RATE = 1e-4
img_rows , img_cols = 40,20
img_channels = 4 #We stack 4 frames

In [9]:
def buildmodel():
    model = Sequential()
    model.add(Conv2D(32, (8, 8), strides=(4, 4), padding='same',input_shape=(img_cols,img_rows,img_channels)))  #20*40*4
    model.add(Activation('relu'))
    model.add(Conv2D(64, (4, 4), strides=(2, 2), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3), strides=(1, 1), padding='same'))
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(ACTIONS))
    adam = Adam(lr=LEARNING_RATE)
    model.compile(loss='mse',optimizer=adam)
    return model

In [10]:
buildmodel().summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 5, 10, 32)         8224      
                                                                 
 activation (Activation)     (None, 5, 10, 32)         0         
                                                                 
 conv2d_1 (Conv2D)           (None, 3, 5, 64)          32832     
                                                                 
 activation_1 (Activation)   (None, 3, 5, 64)          0         
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 5, 64)          36928     
                                                                 
 activation_2 (Activation)   (None, 3, 5, 64)          0         
                                                                 
 flatten (Flatten)           (None, 960)               0

C:\ProgramData\Anaconda3\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [11]:
global game_state
SCREEN_HEIGHT = 800
SCREEN_WIDTH = 400
player = Player((SCREEN_WIDTH/2),(SCREEN_HEIGHT - 50) , 64 , 90)
finishline = FinishLine(SCREEN_WIDTH/2 , 200 , SCREEN_WIDTH, 3)
curator = Curator((SCREEN_WIDTH/2) - (50/2) , 50, 50, 100 , player)
def playGame(observe=False):
    gamestate = GameState.ACTIVE
    game_state = gamestate

    ground_image = pygame.image.load("resources/background.jpg")

    pygame.init()

    clock = pygame.time.Clock()
    FPS = 50


    display_surface = pygame.display.set_mode((SCREEN_WIDTH,SCREEN_HEIGHT))

    pygame.display.set_caption("Red Light, Green Light")


    all_entities = pygame.sprite.Group()
    all_entities.add(player)
    all_entities.add(finishline)
    all_entities.add(curator)
    while True:
        for event in pygame.event.get():
            if event.type == QUIT or keyboard.is_pressed('q') :
                pygame.quit()
                sys.exit()
        display_surface.blit(ground_image,(0,0))

        for entity in all_entities:
            entity.display(display_surface)
        if gamestate == GameState.ACTIVE:
            player.move()
            curator.update()
            if curator.detect_movement():
                print ("You LOST!!!")
                gamestate = GameState.LOST
                game_state = gamestate
                model = buildmodel()
                try:
                    trainNetwork(model,game_state,observe=observe)
                except StopIteration:
                    game.end()
                playGame()
            elif finishline.check_game_won(player.rect.bottom):
                gamestate = GameState.WON
                game_state = gamestate
                print ("YOU WON")
                model = buildmodel()
                try:
                    trainNetwork(model,game_state,observe=observe)
                except StopIteration:
                    game.end()
        player.move()
        pygame.display.update()
        clock.tick(FPS)
        model = buildmodel()
        try:
            trainNetwork(model,game_state,observe=observe)
        except StopIteration:
            game.end()
def get_state(actions,gamestate):
    actions_df.loc[len(actions_df)] = actions[1] 
    display = show_img()
    reward = 10
    if gamestate == GameState.ACTIVE:
        is_over = False 
        if actions[1] == 1:
            reward = 10
            player.agent_move()
    if gamestate == GameState.LOST:
        reward = -10
        is_over = True
        playGame()
    if gamestate == GameState.WON:
        is_over = False 
        if actions[1] == 1:
            reward = 10
    image = grab_screen()
    return image, reward, is_over
def trainNetwork(model,gamestate,observe=True):
    last_time = time.time()
    D = load_obj("D") 
    actions = np.zeros(ACTIONS)
    actions[1] = 1      #0 => Do Nothing,
                        #1=> Move Forward
    x_t, r_0, terminal = get_state(actions,gamestate) 
    s_t = np.stack((x_t, x_t, x_t, x_t), axis=2) 
    s_t = s_t.reshape(1, s_t.shape[0], s_t.shape[1], s_t.shape[2])  
    initial_state = s_t 

    if observe :
        OBSERVE = 999999999    
        epsilon = FINAL_EPSILON
        print ("Now we load weight")
        adam = Adam(lr=LEARNING_RATE)
        model.compile(loss='mse',optimizer=adam)
        print ("Weight load successfully")    
    else:                       #We go to training mode
        OBSERVE = OBSERVATION
        epsilon = load_obj("epsilon") 
        model.load_weights("model_final.h5")
        adam = Adam(lr=LEARNING_RATE)
        model.compile(loss='mse',optimizer=adam)     

In [12]:
def save_obj(obj, name ):
    with open('objects/'+ name + '.pkl', 'wb') as f: 
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
def load_obj(name):
    if os.path.getsize('objects/' + name + '.pkl') > 0:      
        with open('objects/' + name + '.pkl', 'rb') as f:
            return pickle.load(f)
def grab_screen():
    screen =  np.array(ImageGrab.grab(bbox=(40,180,440,400))) 
    image = process_img(screen)
    return image
def process_img(image):
    image = cv2.resize(image, (0,0), fx = 0.15, fy = 0.10) 
    image = image[2:38,10:50] 
    image = cv2.Canny(image, threshold1 = 100, threshold2 = 200)
    return  image
def show_img(graphs = False):
    """
    Show images in new window
    """
    while True:
        screen = (yield)
        window_title = "logs" if graphs else "game_play"
        cv2.namedWindow(window_title, cv2.WINDOW_NORMAL)        
        imS = cv2.resize(screen, (800, 400)) 
        cv2.imshow(window_title, screen)
        if (cv2.waitKey(1) & 0xFF == ord('q')):
            cv2.destroyAllWindows()
            break

In [13]:
loss_file_path = "loss.csv"
actions_file_path = "action.csv"
loss_df = pd.read_csv(loss_file_path) if os.path.isfile(loss_file_path) else pd.DataFrame(columns =['loss'])
actions_df = pd.read_csv(actions_file_path) if os.path.isfile(actions_file_path) else pd.DataFrame(columns = ['actions'])

In [14]:
def init_cache():
    """initial variable caching, done only once"""
    save_obj(INITIAL_EPSILON,"epsilon")
    t = 0
    save_obj(t,"time")
    D = deque()
    save_obj(D,"D")

In [ ]:
playGame(observe=True);

Now we load weight
Weight load successfully
Now we load weight
Weight load successfully
Now we load weight
Weight load successfully
Now we load weight
Weight load successfully
Now we load weight
Weight load successfully
Now we load weight
Weight load successfully
Now we load weight
Weight load successfully
Now we load weight
Weight load successfully
Now we load weight
Weight load successfully
Now we load weight
Weight load successfully
Now we load weight
Weight load successfully
Now we load weight
Weight load successfully
Now we load weight
Weight load successfully
You LOST!!!
You LOST!!!
You LOST!!!
You LOST!!!
You LOST!!!
You LOST!!!
You LOST!!!
You LOST!!!
You LOST!!!
You LOST!!!
You LOST!!!
You LOST!!!
You LOST!!!
You LOST!!!
You LOST!!!
You LOST!!!
You LOST!!!
You LOST!!!
You LOST!!!
You LOST!!!
You LOST!!!
You LOST!!!
You LOST!!!
You LOST!!!
You LOST!!!
You LOST!!!
You LOST!!!
You LOST!!!
You LOST!!!
You LOST!!!
You LOST!!!
You LOST!!!
You LOST!!!
You LOST!!!
You LOST!!!
You LOST